# Deterministic Uncertainty Sets - Multiple uncertain terms, box uncertainty

In [1]:
import cvxpy as cp
import scipy as sc
from sklearn import datasets
import numpy as np
import numpy.random as npr
import numpy.testing as npt
import torch
import pandas as pd
import lropt
import matplotlib.pyplot as plt
RTOL = 1e-04
ATOL = 1e-04
import warnings
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive"]

We solve the problem
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & P(Au+ b)^Tx  \leq  10 + 2(Au + b)^Tx\\
& x \geq 0,\quad x \leq 1,
\end{array}
$$
where again $c$, $P$, and $a$ are constants, and $u$ is the uncertain parameter from a box uncertainty set. We note that the uncertain parameter can appear more than once in the uncertain constraint. 

Note that using the box uncertainty set is equivalent to using the ellipsoidal uncertainty set with $p = \infty$

In [2]:
#formulate using the box set
m = 5
box_u = lropt.UncertainParameter(m, 
                                 uncertainty_set = lropt.Box(rho=2.))
n = 5

# formulate cvxpy variable
x_box = cp.Variable(n)

# formulate problem constants
P = 3*np.random.rand(n,m)
c = np.random.rand(n)
center = 0.5*np.ones(m)
side = 0.1*np.array([1,2,3,4,5])
A = np.diag(0.5*side)
b = center

# formulate objective
objective = cp.Minimize(-c@x_box)

# formulate constraints
constraints = [(P@A@box_u+ P@b)@ x_box - 2*(A@box_u + b)@x_box <= 10 , x_box >=0, x_box<=1]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
newprob = prob_robust.dualize_constraints()
newprob.solve()
print("Box objective value: ", newprob.objective.value, "\nBox x: ", x_box.value)

#formulate using the ellipsoidal set
ellip_u = lropt.UncertainParameter(m,
                                   uncertainty_set = lropt.Ellipsoidal(p = np.inf, 
                                                                       rho=2., a = np.eye(m)))
n = 5

# formulate cvxpy variable
x_el = cp.Variable(n)

# formulate objective
objective = cp.Minimize(-c@x_el)

# formulate constraints
constraints = [P@(A@ellip_u + b)@ x_el  <= 10+(2*A@ellip_u+ 2*b)@x_el, x_el >=0, x_el<=1]

# formulate Robust Problem
prob_robust = lropt.RobustProblem(objective, constraints)

# solve
prob_robust.solve()

print("Ellipsoidal objective value: ", prob_robust.objective.value, "\nEllipsoidal x: ", x_el.value)

# assert x values are equal
npt.assert_allclose(x_box.value, x_el.value, rtol=RTOL, atol=ATOL)


Box objective value:  -1.400371062641744 
Box x:  [1.74966246e-11 1.00000000e+00 9.86373819e-01 2.07932189e-10
 1.33398221e-01]
Ellipsoidal objective value:  -1.4003710626417445 
Ellipsoidal x:  [1.74965509e-11 1.00000000e+00 9.86373819e-01 2.07931738e-10
 1.33398221e-01]


We compare the above with its explicit reformulation and solution
$$
\begin{array}{ll}
\text{minimize} & c^Tx\\
\text{subject to}  & (Pb)^Tx - 2b^Tx + \rho\|A^TP^Tx - 2A^Tx\|_1 \leq  10,\\
\end{array}
$$

In [3]:
# formulate using cvxpy
x_cvxpy = cp.Variable(5)
# formulate objective
objective = cp.Minimize(-c@x_cvxpy)

# formulate constraints
constraints = [(P@b)@x_cvxpy - 2*b@x_cvxpy + 2*cp.norm((P@A).T@x_cvxpy - 2*A.T@x_cvxpy,p=1)<= 10, x_cvxpy>=0, x_cvxpy<=1]

# formulate problem
prob_cvxpy = cp.Problem(objective, constraints)

# solve
prob_cvxpy.solve()

print("Cvxpy objective value: ", prob_cvxpy.objective.value, "\nCvxpy x: ", x_cvxpy.value)

# assert x values are equal
npt.assert_allclose(x_box.value, x_cvxpy.value, rtol=RTOL, atol=ATOL)


Cvxpy objective value:  -1.4003710625915644 
Cvxpy x:  [6.89827876e-11 1.00000000e+00 9.86373818e-01 2.56361977e-09
 1.33398219e-01]
